In [1]:
import warnings
import itertools
import lime_explainer as limee
warnings.filterwarnings(action='ignore')

/usr/local/envs/py2env/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
feature_names = ['customer_id', 'monetary_dnn', 'monetary_btyd', 'frequency_dnn',
                 'frequency_btyd', 'recency', 'T', 'time_between',
                 'avg_basket_value', 'avg_basket_size', 'cnt_returns',
                 'has_returned', 'frequency_btyd_clipped', 'monetary_btyd_clipped',
                 'target_monetary_clipped', 'target_monetary']

In [3]:
explainer = limee.Explainer()

In [4]:
explainer.load_data(
  gcs_path = 'gs://ml-clv-data/model_data.csv',
  features_to_use = [1,3,5,6,7,8,9,10,11],
  feature_names = feature_names,
  categorical_features = [],
  skip_rows=1
)

In [5]:
explainer.create_cmle_client(
  gcp_project = 'ml-clv',
  gcp_model = 'dnn_airflow',
  gcp_model_version = 'v1',
  output_func = lambda x : x['predicted_monetary'] 
)

In [9]:
record, df = explainer.explain_random_record(numeric_rows=[1,2,3,7,8])
print(record)
print(df)

Intercept 4036.5422424835433
Prediction_local [2254.77946306]
Right: 1663.812255859375
[1327.6399999999999, 25.0, 256.0, 256.0, 10.24, 53.11, 31.52, 0.0, 0.0]
                      representation       weight
0                time_between > 8.22  4019.660974
1                        T <= 306.00 -2883.262950
2  1167.98 < monetary_dnn <= 2423.27 -2604.674506
3  44.28 < avg_basket_value <= 66.77 -1170.945605
4               has_returned <= 0.00   857.459308


In [12]:
mod_record = ','.join(['2000'] + [str(e) for e in record[1:]])
print('Mod Record: {}\n\n'.format(mod_record))
explainer.explain_record(mod_record)

Mod Record: 2000,25.0,256.0,256.0,10.24,53.11,31.52,0.0,0.0


Intercept 12188.809896528408
Prediction_local [3172.30544845]
Right: 2114.13818359375


([2000.0, 25.0, 256.0, 256.0, 10.24, 53.11, 31.52, 0.0, 0.0],
                       representation       weight
 0                cnt_returns <= 0.00 -7559.267706
 1                time_between > 8.22 -4756.648567
 2  44.28 < avg_basket_value <= 66.77  4648.972389
 3  1167.98 < monetary_dnn <= 2423.27 -3732.087015
 4               has_returned <= 0.00  2382.526451)